In [1]:
import ifcopenshell
import ifcopenshell.api
import pandas as pd

In [2]:
ifc_path_in = '/Users/fnoi/Library/CloudStorage/OneDrive-TUM/2025_egice_noichl_forth/all_beams.ifc'
ifc_path_out = '/Users/fnoi/Library/CloudStorage/OneDrive-TUM/2025_egice_noichl_forth/auto_test_2.ifc'
ifc_path_out_fix = '/Users/fnoi/Library/CloudStorage/OneDrive-TUM/2025_egice_noichl_forth/auto_test_2_fix.ifc'

In [3]:
# load epea csv file
epea_path = '/Users/fnoi/Library/CloudStorage/OneDrive-TUM/2025_egice_noichl_forth/EPEA_reduce.csv'
# load to pd frame
epea = pd.read_csv(epea_path, names=['madaster_uid', 'external_id', 'cs_type', 'match_2'])
# drop match_2 column
epea = epea.drop('match_2', axis=1)

# remove asterisks and spaces from cs_type
epea['cs_type'] = epea['cs_type'].str.replace('*', '').str.replace(' ', '')


In [ ]:
ifc_file = ifcopenshell.open(ifc_path_in)

# Get all beam type objects
beam_types = ifc_file.by_type("IfcBeamType")
print(f"Found {len(beam_types)} beam type objects")

# Get owner history if it exists
owner_history = None
if ifc_file.by_type("IfcOwnerHistory"):
    owner_history = ifc_file.by_type("IfcOwnerHistory")[0]

# Iterate over each beam type
for beam_type in beam_types:
    beam_name = beam_type.Name if beam_type.Name else f"Unnamed-{beam_type.id()}"
    print(f"Processing beam type: {beam_name}")
    

    profiles = {}
    epea_name = beam_name.split('_')[1]
    print(f'looking for {epea_name}')

    deb = epea[epea.cs_type == epea_name]
    if deb.empty:
        print(f"  No match found for {beam_name}, setting MaterialOrProductId to 'Unknown'")
        deb = 'Unknown'
    else:
        deb = deb['madaster_uid'].values[0]
    print(f"  Found MaterialOrProductId: {deb}")

    # Check if the beam already has Pset_Common
    existing_pset = None
    for rel in ifc_file.by_type("IfcRelDefinesByProperties"):
        if rel.RelatedObjects and beam_type in rel.RelatedObjects:
            if hasattr(rel.RelatingPropertyDefinition, "Name") and rel.RelatingPropertyDefinition.Name == "Pset_Common":
                existing_pset = rel.RelatingPropertyDefinition
                break

    # Create or use existing Pset_Common
    if existing_pset:
        pset = existing_pset
        print(f"  Using existing Pset_Common for {beam_name}")
    else:
        # Create a new property set
        pset = ifc_file.create_entity(
            "IfcPropertySet",
            GlobalId=ifcopenshell.guid.new(),
            OwnerHistory=owner_history,
            Name="Pset_Common",
            Description=None,
            HasProperties=[]
        )

        # Create relationship to beam type
        rel = ifc_file.create_entity(
            "IfcRelDefinesByProperties",
            GlobalId=ifcopenshell.guid.new(),
            OwnerHistory=owner_history,
            RelatedObjects=[beam_type],
            RelatingPropertyDefinition=pset
        )
        print(f"  Created new Pset_Common for {deb}")

    # Check if MaterialOrProductId already exists
    existing_property = None
    if hasattr(pset, "HasProperties"):
        for prop in pset.HasProperties:
            if prop.Name == "MaterialOrProductId":
                existing_property = prop
                break

    # Add or update MaterialOrProductId property
    if existing_property:
        # Update existing property
        if hasattr(existing_property, "NominalValue"):
            existing_property.NominalValue.wrappedValue = deb
            print(f"  Updated MaterialOrProductId: {deb}")
    else:
        # Create new property
        new_prop = ifc_file.create_entity(
            "IfcPropertySingleValue",
            Name="MaterialOrProductId",
            Description="",
            NominalValue=ifc_file.create_entity("IfcText", deb)
        )

        # Add property to property set
        props = list(pset.HasProperties) if hasattr(pset, "HasProperties") and pset.HasProperties else []
        props.append(new_prop)
        pset.HasProperties = props
        print(f"  Added MaterialOrProductId: {deb}")

    # Check if DetachabilityConnectionType already exists
    existing_detach_property = None
    if hasattr(pset, "HasProperties"):
        for prop in pset.HasProperties:
            if prop.Name == "DetachabilityConnectionType":
                existing_detach_property = prop
                break

    # Add or update DetachabilityConnectionType property
    if existing_detach_property:
        # Update existing property
        if hasattr(existing_detach_property, "NominalValue"):
            existing_detach_property.NominalValue.wrappedValue = "HardChemicalConnection"
            print(f"  Updated DetachabilityConnectionType: HardChemicalConnection")
    else:
        # Create new property
        new_detach_prop = ifc_file.create_entity(
            "IfcPropertySingleValue",
            Name="DetachabilityConnectionType",
            Description="Type of connection for detachability assessment",
            NominalValue=ifc_file.create_entity("IfcText", "HardChemicalConnection")
        )

        # Add property to property set
        props = list(pset.HasProperties) if hasattr(pset, "HasProperties") and pset.HasProperties else []
        props.append(new_detach_prop)
        pset.HasProperties = props
        print(f"  Added DetachabilityConnectionType: HardChemicalConnection")

# Save the modified file
ifc_file.write(ifc_path_out)
print(f"Saved modified file as: auto_test.ifc")

Found 14 beam type objects
Processing beam type: T_HEA140
looking for HEA140
  Found MaterialOrProductId: 6a04ca4b-3bee-4717-b82c-4cadcfb5f187
  Created new Pset_Common for 6a04ca4b-3bee-4717-b82c-4cadcfb5f187
  Added MaterialOrProductId: 6a04ca4b-3bee-4717-b82c-4cadcfb5f187
  Added DetachabilityConnectionType: HardChemicalConnection
Processing beam type: T_HEA160
looking for HEA160
  Found MaterialOrProductId: 9e9d415f-29fb-496a-8e6d-b20104079f01
  Created new Pset_Common for 9e9d415f-29fb-496a-8e6d-b20104079f01
  Added MaterialOrProductId: 9e9d415f-29fb-496a-8e6d-b20104079f01
  Added DetachabilityConnectionType: HardChemicalConnection
Processing beam type: T_HEA200
looking for HEA200
  Found MaterialOrProductId: d643749d-d2e4-420a-82a7-e7f087963a2a
  Created new Pset_Common for d643749d-d2e4-420a-82a7-e7f087963a2a
  Added MaterialOrProductId: d643749d-d2e4-420a-82a7-e7f087963a2a
  Added DetachabilityConnectionType: HardChemicalConnection
Processing beam type: T_HEA240
looking for HEA

In [6]:
# Instead of checking for Pset_Common, look for CPset_Madaster
existing_pset = None
for rel in ifc_file.by_type("IfcRelDefinesByProperties"):
    if rel.RelatedObjects and beam_type in rel.RelatedObjects:
        if hasattr(rel.RelatingPropertyDefinition, "Name") and rel.RelatingPropertyDefinition.Name == "CPset_Madaster":
            existing_pset = rel.RelatingPropertyDefinition
            break

# Create CPset_Madaster if it doesn't exist
if existing_pset:
    pset = existing_pset
    print(f"  Using existing CPset_Madaster for {beam_name}")
else:
    # Create a new property set with Madaster name
    pset = ifc_file.create_entity(
        "IfcPropertySet",
        GlobalId=ifcopenshell.guid.new(),
        OwnerHistory=owner_history,
        Name="CPset_Madaster",
        Description=None,
        HasProperties=[]
    )
    
    # Create relationship to beam type
    rel = ifc_file.create_entity(
        "IfcRelDefinesByProperties",
        GlobalId=ifcopenshell.guid.new(),
        OwnerHistory=owner_history,
        RelatedObjects=[beam_type],
        RelatingPropertyDefinition=pset
    )
    print(f"  Created new CPset_Madaster for {deb}")

# Create properties dictionary
madaster_properties = {
    "MaterialOrProductId": deb,
    "MaterialOrProductName": f"Steel Beam {epea_name}",
    "MaterialOrProductRatio": "1",
    "DetachabilityConnectionTypeDetail": "HardChemicalConnection",
    "DetachabilityIntersection": "None",
    "DetachabilityProductEdge": "Overlapping",
    "DetachabilityAccessibility": "Accessible",
    "Classification": "351",
    "Phase": "Bestand"
}

# Add or update all properties
props = list(pset.HasProperties) if hasattr(pset, "HasProperties") and pset.HasProperties else []
existing_prop_names = [prop.Name for prop in props] if props else []

for prop_name, prop_value in madaster_properties.items():
    # Check if property already exists
    if prop_name in existing_prop_names:
        # Update existing property
        for prop in props:
            if prop.Name == prop_name:
                prop.NominalValue.wrappedValue = prop_value
                print(f"  Updated {prop_name}: {prop_value}")
                break
    else:
        # Create new property
        new_prop = ifc_file.create_entity(
            "IfcPropertySingleValue",
            Name=prop_name,
            Description="",
            NominalValue=ifc_file.create_entity("IfcText", prop_value)
        )
        props.append(new_prop)
        print(f"  Added {prop_name}: {prop_value}")

# Update property set with all properties
pset.HasProperties = props

ifc_file.write(ifc_path_out_fix)
print(f"Saved modified file as: auto_test_fix.ifc")

  Using existing CPset_Madaster for T_HEM180
  Updated MaterialOrProductId: 81834ca6-d459-4671-864a-52c4259e5616
  Updated MaterialOrProductName: Steel Beam HEM180
  Updated MaterialOrProductRatio: 1
  Updated DetachabilityConnectionTypeDetail: HardChemicalConnection
  Updated DetachabilityIntersection: None
  Updated DetachabilityProductEdge: Overlapping
  Updated DetachabilityAccessibility: Accessible
  Updated Classification: 351
  Updated Phase: Bestand
Saved modified file as: auto_test_fix.ifc
